In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import trimesh

scene = trimesh.Scene()

In [2]:
import my_code.diffusion_training_sign_corr.data_loading as data_loading

train_dataset = data_loading.get_val_dataset(
    'FAUST_r', 'train', 200, canonicalize_fmap=None
    )[1]
test_dataset = data_loading.get_val_dataset(
    'FAUST_r', 'test', 200, canonicalize_fmap=None
)[1]

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Loading base dataset: 100%|████████████████████████████████████████████████████████████████████████████████| 20/20 [00:04<00:00,  4.61it/s]


In [8]:
import networks.diffusion_network as diffusion_network

condition_dim = 0
start_dim = 0

feature_dim = 32
evecs_per_support = 4

device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = diffusion_network.DiffusionNet(
    in_channels=feature_dim,
    out_channels=feature_dim // evecs_per_support,
    cache_dir=None,
    input_type='wks',
    k_eig=64,
    n_block=6,
    # diffusion_method='implicit_dense'
    ).to(device)

In [6]:
from tqdm import tqdm

train_diff_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_r/diffusion'
test_diff_folder = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_r/diffusion'

net.cache_dir = train_diff_folder

for i in tqdm(range(len(train_dataset))):
    data_second = train_dataset[i]['second']
    verts = data_second['verts'].unsqueeze(0).to(device)
    faces = data_second['faces'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        net(verts, faces)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:03<00:00, 23.13it/s]


In [9]:

net.cache_dir = '/home/s94zalek_hpc/shape_matching/notebooks/31.07.2024/64'

for i in tqdm(range(len(train_dataset))):
    data_second = train_dataset[i]['second']
    verts = data_second['verts'].unsqueeze(0).to(device)
    faces = data_second['faces'].unsqueeze(0).to(device)
    
    with torch.no_grad():
        net(verts, faces)

 11%|███████████▌                                                                                           | 9/80 [00:05<00:46,  1.53it/s]


KeyboardInterrupt: 

In [19]:
import os

target_dir = '/home/s94zalek_hpc/shape_matching/notebooks/31.07.2024/64'
for filename in os.listdir(target_dir):
    print(f"{os.path.getsize(target_dir + '/' + filename) / 1024 / 1024:.2f} MB")
    
# target_dir = '/home/s94zalek_hpc/shape_matching/notebooks/31.07.2024/128'
# for filename in os.listdir(target_dir):
#     print(f"{os.path.getsize(target_dir + '/' + filename) / 1024 / 1024:.2f} MB")

# target_dir = '/home/s94zalek_hpc/shape_matching/data_with_smpl_corr/FAUST_r/diffusion'
# for filename in os.listdir(target_dir):
#     print(f"{os.path.getsize(target_dir + '/' + filename) / 1024 / 1024:.2f} MB")

2.56 MB
2.56 MB
2.56 MB
2.56 MB
2.56 MB
2.56 MB
2.56 MB
2.56 MB
2.56 MB
